# Interactive Visualization Lab

Complete the following set of exercises to solidify your knowledge of interactive visualization using Plotly, Cufflinks, and IPyWidgets.

In [ ]:
# Este código da error; empezar a partir de la siguiente celda.

import pandas as pd
import plotly.plotly as py
import cufflinks as cf
from ipywidgets import interact

cf.go_offline()

#ImportError: 
#The plotly.plotly module is deprecated,
#please install the chart-studio package and use the
#chart_studio.plotly module instead. 

In [1]:
import pandas as pd
import chart_studio.plotly as py
import cufflinks as cf
from ipywidgets import interact
from ipywidgets import interact_manual
import ipywidgets as widgets

cf.go_offline()

In [2]:
data = pd.read_excel('../data/Online Retail.xlsx')

In [3]:
data.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
0,536365,2010-12-01 08:26:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
1,536373,2010-12-01 09:02:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
2,536375,2010-12-01 09:32:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
3,536390,2010-12-01 10:19:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,64,2.55,163.2,17511,United Kingdom
4,536394,2010-12-01 10:39:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,32,2.55,81.6,13408,United Kingdom


## 1. Create an interactive bar chart showing total quantity and revenue by country (excluding United Kingdom) for the month of April 2011.

In [4]:
df = data.copy()

In [5]:
df['Year'] = df['InvoiceDate'].astype(str).apply(lambda string: string[0:4]).astype(int)
df['#Month'] = df['InvoiceDate'].astype(str).apply(lambda string: string[5:7]).astype(int)
df.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country,Year,#Month
0,536365,2010-12-01 08:26:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom,2010,12
1,536373,2010-12-01 09:02:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom,2010,12
2,536375,2010-12-01 09:32:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom,2010,12
3,536390,2010-12-01 10:19:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,64,2.55,163.2,17511,United Kingdom,2010,12
4,536394,2010-12-01 10:39:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,32,2.55,81.6,13408,United Kingdom,2010,12


In [102]:
# Podría filtrar por abril de 2011, pero prefiero crear dos decoradores para hacer el filtro por año y mes.
# No he conseguido poner los decoradores con abril de 2011 por defecto (me saca diciembre de 2010 por defecto).

@interact(Year=list(df['Year'].unique()), 
          Month=list(df['#Month'].unique())
         )

def barchart(Year, Month):
    df_chart = df[(df['Country'] != 'United Kingdom') & (df['Year']==Year) & (df['#Month']==Month)].groupby('Country').agg({'Quantity':'sum', 'Revenue':'sum'}).reset_index()

    df_chart.iplot(kind='bar', x='Country', xTitle='Country',
               yTitle='Total', color=['blue', 'red'], 
               title='Quantity and revenue by country')

interactive(children=(Dropdown(description='Year', options=(2010, 2011), value=2010), Dropdown(description='Mo…

## 2. Create an interactive line chart showing quantity and revenue sold to France between January 1st and May 31st 2011.

In [7]:
# Creo el campo Date, que es como InvoiceDate pero solo año, mes y día. Para usarlo como filtro en el decorador.
df['Date'] = pd.DatetimeIndex(data['InvoiceDate']).date

In [103]:
@interact(Start_date=list(df['Date'].unique()), 
          End_date=list(df['Date'].unique())
         )

def linechart(Start_date, End_date):
    df_chart = df[(df['Country'] == 'France') & (df['Date'] >= Start_date) & (df['Date'] >= End_date)].groupby('InvoiceDate').agg({'Quantity':'sum', 'Revenue':'sum'}).reset_index()

    df_chart.iplot(kind='line', x='InvoiceDate', xTitle='France',
               yTitle='Total', color=['blue', 'red'], 
               title='Quantity and revenue sold to France')

interactive(children=(Dropdown(description='Start_date', options=(datetime.date(2010, 12, 1), datetime.date(20…

In [104]:
# Lo hago de otra forma que he visto, con calendario y sin necesidad de crear la columna Date

@interact(Start_date=widgets.DatePicker(value=pd.to_datetime('2011-01-01')), 
          End_date=widgets.DatePicker(value=pd.to_datetime('2011-05-31'))
         )

def linechart(Start_date, End_date):
    df_chart = df[(df['Country'] == 'France') & (df['InvoiceDate'] >= Start_date) & (df['InvoiceDate'] >= End_date)].groupby('InvoiceDate').agg({'Quantity':'sum', 'Revenue':'sum'}).reset_index()

    df_chart.iplot(kind='line', x='InvoiceDate', xTitle='France',
               yTitle='Total', color=['blue', 'red'], 
               title='Quantity and revenue sold to France')

interactive(children=(DatePicker(value=Timestamp('2011-01-01 00:00:00'), description='Start_date'), DatePicker…

## 3. Create an interactive scatter plot showing the relationship between average quantity (x-axis) and average unit price (y-axis) for the product PARTY BUNTING with the plot points color-coded by country (categories).

In [10]:
# Quiero ver qué productos contiene la cadena 'PARTY BUNTING'
df[df['Description'].str.contains('PARTY BUNTING') == True]['Description'].unique()

array(['TEA TIME PARTY BUNTING', 'PARTY BUNTING'], dtype=object)

In [100]:
@interact_manual(Product='PARTY BUNTING')

def scatter(Product):
    try:
        df_chart = df[df['Description']==Product.upper()].groupby('Country').agg({'Quantity':'mean', 'UnitPrice':'mean'}).reset_index()

        df_chart.iplot(kind='scatter', x='Quantity', y='UnitPrice', categories='Country',
                    xTitle='Quantity', yTitle='UnitPrice', 
                    title='Avg Quantity vs Avg Unit Price')

    except:
        print("The description of the product is not correct. Please enter the description again.")

interactive(children=(Text(value='PARTY BUNTING', description='Product'), Button(description='Run Interact', s…

## 4. Create a set of interactive histograms showing the distributions of quantity per invoice for the following countries: EIRE, Germany, France, and Netherlands.

In [105]:
@interact(Country=['EIRE', 'Germany', 'France', 'Netherlands'])
def hist(Country):
    df_chart = df[df['Country']==Country].groupby('InvoiceNo').agg({'Quantity':'sum'}).reset_index()
    df_chart['Quantity'].iplot(kind='hist', bins=40, title='Quantity per invoice',
                                xTitle='Quantity of items per invoice', yTitle='Number of orders')

interactive(children=(Dropdown(description='Country', options=('EIRE', 'Germany', 'France', 'Netherlands'), va…

El histograma anterior significa que para EIRE hay:
* 79 pedidos con entre 0 y 199 artículos
* 72 pedidos con entre 200 y 399 artículos
* Etc.

En los 4 países la distribución es prácticamente similar.

## 5. Create an interactive side-by-side bar chart showing the revenue by country listed below (bars) for each of the products listed below.

In [31]:
product_list = ['JUMBO BAG RED RETROSPOT', 
                'CREAM HANGING HEART T-LIGHT HOLDER',
                'REGENCY CAKESTAND 3 TIER']

country_list = ['EIRE', 'Germany', 'France', 'Netherlands']

In [106]:
@interact(Product=product_list)

def barchart(Product):
    df_chart = df[(df['Country'].isin(country_list)) & (df['Description']==Product)].groupby('Country').agg({'Revenue':'sum'}).reset_index()

    df_chart.iplot(kind='bar', x='Country', xTitle='Country',
               yTitle='Revenue', color='blue', 
               title='Revenue by country')
    

interactive(children=(Dropdown(description='Product', options=('JUMBO BAG RED RETROSPOT', 'CREAM HANGING HEART…

## 6. Create an interactive line chart showing quantity sold by day for the United Kingdom. Add drop-down boxes for Year and Month that allow you to filter the date range that appears in the chart.

In [ ]:
data['Year'] = pd.DatetimeIndex(data['InvoiceDate']).year
data['Month'] = pd.DatetimeIndex(data['InvoiceDate']).month
data['Day'] = pd.DatetimeIndex(data['InvoiceDate']).day
uk = data[data['Country']=='United Kingdom']

In [44]:
uk.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country,Year,Month,Day
0,536365,2010-12-01 08:26:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom,2010,12,1
1,536373,2010-12-01 09:02:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom,2010,12,1
2,536375,2010-12-01 09:32:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom,2010,12,1
3,536390,2010-12-01 10:19:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,64,2.55,163.2,17511,United Kingdom,2010,12,1
4,536394,2010-12-01 10:39:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,32,2.55,81.6,13408,United Kingdom,2010,12,1


In [107]:
@interact(Year=list(uk['Year'].unique()), 
          Month=list(uk['Month'].unique())
         )

def linechart(Year, Month):
    df_chart = uk[(uk['Year']==Year) & (uk['Month']==Month)].groupby('Day').agg({'Quantity':'sum'}).reset_index()

    df_chart.iplot(kind='line', x='Day', xTitle='Country',
               yTitle='Total', color=['blue', 'red'], 
               title='Quantity and revenue by country')

interactive(children=(Dropdown(description='Year', options=(2010, 2011), value=2010), Dropdown(description='Mo…

## 7. Create an interactive scatter plot that plots number of invoices (x-axis) vs. number of customers (y-axis) and the plot points represent individual products. Add two sliders that control the x and y axis ranges.

In [ ]:
agg_func = {'InvoiceNo':'nunique',
            'Quantity':'sum',
            'UnitPrice':'mean',
            'Revenue':'sum',
            'CustomerID':'nunique'}

products = uk.groupby('Description').agg(agg_func)

In [84]:
# No he conseguido vincular el slider con el gráfico.
# No encuentro por ningún sitio un xlim/ylim a los que vincular los valores de Invoices y Customers.

@interact(Invoices = (0,len(df['InvoiceNo'].unique())),
         Customers = (0,len(df['CustomerID'].unique()))
         )

def scatter(Invoices, Customers):
    df_chart = df.groupby('Description').agg({'InvoiceNo':'nunique', 'CustomerID':'nunique'}).reset_index()

    df_chart.iplot(kind='scatter', x='InvoiceNo', y='CustomerID', categories='Description',
                xTitle='Number of invoices', yTitle='Number of customers', 
                title='Number of Invoices vs Number of Customers')

interactive(children=(IntSlider(value=9194, description='Invoices', max=18389), IntSlider(value=2164, descript…

## 8. Creat an interactive bar chart that shows revenue by product description. Add a text field widget that filters the results to show the product that contain the text entered in their description.

In [101]:
@interact_manual(Product='')

def barchart(Product):
    """
    Meto un contains y un upper para facilitar la entrada del producto en el cuadro de texto.
    """
    df_chart = df[df['Description'].str.contains(Product.upper())].groupby('Description').agg({'Revenue':'sum'}).reset_index()

    df_chart.iplot(kind='bar', x='Description', xTitle='Product',
               yTitle='Revenue', color='blue', 
               title='Revenue by product')
        

interactive(children=(Text(value='', description='Product'), Button(description='Run Interact', style=ButtonSt…